In [1]:
from google.colab import files
uploaded = files.upload()

import os, shutil
fname = next(iter(uploaded))  # 첫 파일명
if fname != "kaggle.json":
    shutil.move(fname, "kaggle.json")

!pip -q install kaggle
!mkdir -p ~/.kaggle /root/.config/kaggle
!cp kaggle.json ~/.kaggle/
!cp kaggle.json /root/.config/kaggle/
!chmod 600 ~/.kaggle/kaggle.json /root/.config/kaggle/kaggle.json

!kaggle --version

Saving kaggle.json to kaggle.json
Kaggle API 1.7.4.5


In [2]:
import os
KAGGLE_USERNAME = "kwontaejin"
KAGGLE_KEY = "26103524"
os.environ["kwontaejin"] = KAGGLE_USERNAME
os.environ["26103524"] = KAGGLE_KEY

!pip -q install kaggle

In [3]:
COMP="aikuthon9th"
!kaggle competitions download -c {COMP}
!mkdir -p data
!unzip -q -o {COMP}.zip -d data

!ls -al data | sed -n '1,120p'

100% 4.04G/4.04G [00:38<00:00, 40.8MB/s]
100% 4.04G/4.04G [00:38<00:00, 112MB/s] 
total 392
drwxr-xr-x 4 root root   4096 Aug 29 13:59 .
drwxr-xr-x 1 root root   4096 Aug 29 13:58 ..
drwxr-xr-x 2 root root  40960 Aug 29 13:58 test_images
drwxr-xr-x 2 root root 266240 Aug 29 13:59 train_images
-rw-r--r-- 1 root root  77601 Aug 26 08:28 train_metadata.csv


In [5]:
!pip -q install timm albumentations opencv-python torchmetrics==1.4.0

import os, glob, math, random, cv2, numpy as np, pandas as pd
from collections import Counter

import torch, torch.nn as nn, torch.optim as optim
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler
from torch.cuda.amp import autocast, GradScaler
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score, accuracy_score

import albumentations as A
from albumentations.pytorch import ToTensorV2
import timm
from timm.utils import ModelEmaV2

CFG = dict(
    seed=42,
    folds=5,
    img_size=224,            #  224로 다운스케일해서 사용
    num_classes=6,
    epochs=20,
    batch_size=64,
    num_workers=2,
    accum_steps=2,
    lr=3e-4,
    weight_decay=5e-2,
    warmup_epochs=3,
    ema=True,
    model_name="vit_hybrid_r50_s16_224",
    root="data",
    train_dir_name="train_images",   # 원본 512×512
    test_dir_name="test_images",     # 원본 512×512
    meta_csv="train_metadata.csv",
    cb_beta=0.999, cb_gamma=1.5,     # CB-Focal
    sampler_alpha=0.0,                # oversampling
)

def set_seed(seed=42):
    random.seed(seed); np.random.seed(seed)
    torch.manual_seed(seed); torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.benchmark = True
set_seed(CFG["seed"])

ROOT   = CFG["root"]
TRAIND = os.path.join(ROOT, CFG["train_dir_name"])
TESTD  = os.path.join(ROOT, CFG["test_dir_name"])
META   = os.path.join(ROOT, CFG["meta_csv"])

assert os.path.exists(TRAIND), f"Not found: {TRAIND}"
assert os.path.exists(TESTD),  f"Not found: {TESTD}"
assert os.path.exists(META),   f"Not found: {META}"

EXTS = ["*.png"]

def scan_files(dirpath):
    files=[]
    for ext in EXTS: files += glob.glob(os.path.join(dirpath, ext))
    def sk(p):
        s = os.path.splitext(os.path.basename(p))[0]
        try: return (0, int(s))
        except: return (1, s)
    return sorted(files, key=sk)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 868.8/868.8 kB 22.8 MB/s eta 0:00:00


In [6]:
train_files = scan_files(TRAIND)
train_meta = pd.read_csv(META)

print("Number of training images:", len(train_files))
display(train_meta.head())

Number of training images: 9699


,id,label
0,1,0
1,2,1
2,3,0
3,4,3
4,5,0


# Task
Modify the code in the selected cell to address the class imbalance issue for labels 1, 2, and 5 in the dataset loaded from "/content/train_metadata.csv". The original images are 512x512, but the input size should be 224x224. Implement dropout and mixup techniques and use the efficientnet-b0 model.

## 데이터셋 클래스 불균형 확인

### Subtask:
`train_metadata.csv` 파일에서 각 레이블의 분포를 다시 한번 확인하여 클래스 불균형 정도를 파악합니다.


**Reasoning**:
The `train_meta` DataFrame is already loaded and contains the label information. I need to calculate the value counts for the 'label' column to check the class distribution.



In [7]:
label_counts = train_meta['label'].value_counts().sort_index()
print("Label distribution:")
print(label_counts)

Label distribution:
label
0    3000
1     200
2     499
3    3000
4    2200
5     800
Name: count, dtype: int64


## 데이터 전처리 및 증강

### Subtask:
이미지 크기를 224x224로 조정하고, 클래스 불균형 해소를 위해 오버샘플링 또는 다른 기법을 고려합니다. 또한, dropout과 mixup을 활용한 데이터 증강 기법을 정의합니다.


**Reasoning**:
Define Albumentations transformations for training and validation, and calculate class weights for WeightedRandomSampler.



In [8]:
train_transform = A.Compose([
    A.Resize(CFG["img_size"], CFG["img_size"]),
    A.HorizontalFlip(p=0.5),
    A.Rotate(limit=10, p=0.5),
    A.ColorJitter(brightness=0.1, contrast=0.1, saturation=0.1, hue=0.1, p=0.5),
    A.ShiftScaleRotate(shift_limit=0.05, scale_limit=0.05, rotate_limit=10, p=0.5),
    A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ToTensorV2(),
])

val_transform = A.Compose([
    A.Resize(CFG["img_size"], CFG["img_size"]),
    A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ToTensorV2(),
])

# Calculate class weights for WeightedRandomSampler
class_counts = train_meta['label'].value_counts().sort_index()
class_weights = 1. / class_counts.values
weights = class_weights[train_meta['label'].values]

def mixup_data(x, y, alpha=1.0, num_classes=CFG["num_classes"]):
    '''Compute the mixup data. Return mixed inputs, pairs of targets, and lambda'''
    if alpha > 0.:
        lam = np.random.beta(alpha, alpha)
    else:
        lam = 1.
    batch_size = x.size()[0]
    index = torch.randperm(batch_size).cuda()
    mixed_x = lam * x + (1 - lam) * x[index,:]
    y_a, y_b = y, y[index]
    return mixed_x, y_a, y_b, lam

def mixup_criterion(criterion, pred, y_a, y_b, lam):
    return lam * criterion(pred, y_a) + (1 - lam) * criterion(pred, y_b)

/usr/local/lib/python3.12/dist-packages/albumentations/core/validation.py:114: UserWarning: ShiftScaleRotate is a special case of Affine transform. Please use Affine transform instead.
  original_init(self, **validated_kwargs)


## 모델 정의

### Subtask:
EfficientNet-B0 모델을 로드하고, 데이터셋의 클래스 수에 맞게 모델의 마지막 레이어를 수정합니다. dropout을 모델에 적용합니다.


**Reasoning**:
Load the efficientnet_b0 model, modify its classifier layer for the correct number of classes, and add a dropout layer.



In [9]:
def build_model(model_name=CFG["model_name"], num_classes=CFG["num_classes"], pretrained=True):
    model = timm.create_model(model_name, pretrained=pretrained)
    # Get the number of input features for the classifier
    in_features = model.get_classifier().in_features

    # Add dropout before the classifier
    dropout_rate = CFG.get("dropout_rate", 0.2) # Use default if not in CFG
    model.classifier = nn.Sequential(
        nn.Dropout(p=dropout_rate),
        nn.Linear(in_features, num_classes)
    )

    return model

CFG["model_name"] = "efficientnet_b0" # Update model name in CFG
CFG["dropout_rate"] = 0.2 # Add dropout rate to CFG

model = build_model(model_name=CFG["model_name"], num_classes=CFG["num_classes"])
print(model)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model.safetensors:   0%|          | 0.00/21.4M [00:00<?, ?B/s]

EfficientNet(
  (conv_stem): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
  (bn1): BatchNormAct2d(
    32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True
    (drop): Identity()
    (act): SiLU(inplace=True)
  )
  (blocks): Sequential(
    (0): Sequential(
      (0): DepthwiseSeparableConv(
        (conv_dw): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
        (bn1): BatchNormAct2d(
          32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True
          (drop): Identity()
          (act): SiLU(inplace=True)
        )
        (aa): Identity()
        (se): SqueezeExcite(
          (conv_reduce): Conv2d(32, 8, kernel_size=(1, 1), stride=(1, 1))
          (act1): SiLU(inplace=True)
          (conv_expand): Conv2d(8, 32, kernel_size=(1, 1), stride=(1, 1))
          (gate): Sigmoid()
        )
        (conv_pw): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn2

## 손실 함수 및 옵티마이저 정의

### Subtask:
클래스 불균형을 고려한 손실 함수 (예: Weighted Cross Entropy 또는 CB-Focal)와 옵티마이저를 정의합니다.


**Reasoning**:
Import necessary modules, calculate class weights, and define the loss function and optimizer based on the provided configurations and class counts.



In [10]:
import torch.nn as nn
import torch.optim as optim

# Calculate class weights based on the inverse of the class counts
class_counts = train_meta['label'].value_counts().sort_index()
class_weights = 1. / torch.tensor(class_counts.values, dtype=torch.float)
# Normalize weights
class_weights = class_weights / class_weights.sum()
# Move weights to the appropriate device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
class_weights = class_weights.to(device)

# Define the loss function (Weighted Cross Entropy)
criterion = nn.CrossEntropyLoss(weight=class_weights)

# Define the optimizer
optimizer = optim.AdamW(model.parameters(), lr=CFG["lr"], weight_decay=CFG["weight_decay"])

print("Class weights for loss function:", class_weights)
print("Loss function defined:", criterion)
print("Optimizer defined:", optimizer)

Class weights for loss function: tensor([0.0356, 0.5333, 0.2138, 0.0356, 0.0485, 0.1333], device='cuda:0')
Loss function defined: CrossEntropyLoss()
Optimizer defined: AdamW (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    capturable: False
    decoupled_weight_decay: True
    differentiable: False
    eps: 1e-08
    foreach: None
    fused: None
    lr: 0.0003
    maximize: False
    weight_decay: 0.05
)


## 학습 및 평가 함수 구현

### Subtask:
모델 학습 및 평가를 위한 함수를 구현하고, mixup을 학습 과정에 적용합니다. F1-score와 Accuracy를 평가 지표로 사용합니다.


**Reasoning**:
Implement the training and evaluation functions including mixup and metric calculation as per the instructions.



In [13]:
from torchmetrics.classification import F1Score, Accuracy

def train_one_epoch(model, loader, criterion, optimizer, device, scaler=None, mixup_alpha=0.0):
    model.train()
    running_loss = 0.0

    for i, (images, labels) in enumerate(loader):
        images, labels = images.to(device), labels.to(device)

        if mixup_alpha > 0:
            images, labels_a, labels_b, lam = mixup_data(images, labels, mixup_alpha, CFG["num_classes"])

        with autocast(enabled=scaler is not None):
            outputs = model(images)
            if mixup_alpha > 0:
                loss = mixup_criterion(criterion, outputs, labels_a, labels_b, lam)
            else:
                loss = criterion(outputs, labels)

        if scaler is not None:
            scaler.scale(loss).backward()
        else:
            loss.backward()

        if (i + 1) % CFG["accum_steps"] == 0:
            if scaler is not None:
                scaler.step(optimizer)
                scaler.update()
            else:
                optimizer.step()
            optimizer.zero_grad()

        running_loss += loss.item() * images.size(0)

    epoch_loss = running_loss / len(loader.dataset)
    return epoch_loss

def evaluate_model(model, loader, criterion, device):
    model.eval()
    running_loss = 0.0
    all_preds = []
    all_labels = []

    f1_metric = F1Score(task="multiclass", num_classes=CFG["num_classes"]).to(device)


    with torch.no_grad():
        for images, labels in loader:
            images, labels = images.to(device), labels.to(device)

            outputs = model(images)
            loss = criterion(outputs, labels)

            running_loss += loss.item() * images.size(0)

            preds = torch.argmax(outputs, dim=1)
            all_preds.append(preds)
            all_labels.append(labels)

    epoch_loss = running_loss / len(loader.dataset)
    all_preds = torch.cat(all_preds)
    all_labels = torch.cat(all_labels)

    f1 = f1_metric(all_preds, all_labels)


    return epoch_loss, f1.item()

print("Training and evaluation functions defined.")

Training and evaluation functions defined.


## 모델 학습

### Subtask:
정의된 모델, 손실 함수, 옵티마이저, 데이터 로더를 사용하여 모델을 학습합니다.


**Reasoning**:
Implement the main training loop, including data preparation, model training, evaluation, and saving the best model based on validation performance.



In [18]:
class AIKuthonDataset(Dataset):
    def __init__(self, file_paths, labels=None, train_transform=None, val_transform=None):
        self.file_paths = file_paths
        self.labels = labels
        self.train_transform = train_transform
        self.val_transform = val_transform
        self.augment_labels = [1, 2, 5] # Labels to apply augmentation

    def __len__(self):
        return len(self.file_paths)

    def __getitem__(self, idx):
        img_path = self.file_paths[idx]
        image = cv2.imread(img_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        if self.labels is not None:
            label = self.labels[idx]
            if label in self.augment_labels and self.train_transform:
                image = self.train_transform(image=image)["image"]
            elif self.val_transform:
                 image = self.val_transform(image=image)["image"]
            return image, torch.tensor(label, dtype=torch.long)
        else:
            # Apply validation transform to test images
            if self.val_transform:
                image = self.val_transform(image=image)["image"]
            return image, -1 # Return a placeholder for label

# Prepare data for training and validation
train_files = scan_files(TRAIND)
train_labels = train_meta['label'].values

# Stratified k-fold cross-validation
skf = StratifiedKFold(n_splits=CFG["folds"], shuffle=True, random_state=CFG["seed"])

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

best_f1 = 0.0
best_model_path = "best_model.pth"
early_stopping_patience = 5 # Define early stopping patience
epochs_without_improvement = 0

for fold, (train_index, val_index) in enumerate(skf.split(train_files, train_labels)):
    print(f"===== Fold {fold+1}/{CFG['folds']} =====")

    train_fold_files = [train_files[i] for i in train_index]
    train_fold_labels = [train_labels[i] for i in train_index]
    val_fold_files = [train_files[i] for i in val_index]
    val_fold_labels = [train_labels[i] for i in val_index]

    # Create datasets and data loaders
    train_dataset = AIKuthonDataset(train_fold_files, train_fold_labels, train_transform=train_transform, val_transform=val_transform)
    val_dataset = AIKuthonDataset(val_fold_files, val_fold_labels, val_transform=val_transform)

    # Calculate weights for WeightedRandomSampler based on the current fold's training data
    fold_label_counts = pd.Series(train_fold_labels).value_counts().sort_index()
    fold_class_weights = 1. / fold_label_counts.values
    fold_weights = fold_class_weights[train_fold_labels]
    fold_weights = torch.DoubleTensor(fold_weights)

    train_sampler = WeightedRandomSampler(
        weights=fold_weights,
        num_samples=len(fold_weights),
        replacement=True
    )

    train_loader = DataLoader(
        train_dataset,
        batch_size=CFG["batch_size"],
        sampler=train_sampler,
        num_workers=CFG["num_workers"],
        pin_memory=True,
        drop_last=True,
    )

    val_loader = DataLoader(
        val_dataset,
        batch_size=CFG["batch_size"],
        num_workers=CFG["num_workers"],
        pin_memory=True,
        shuffle=False,
    )

    # Initialize model, criterion, optimizer, and scaler for each fold
    model = build_model(model_name=CFG["model_name"], num_classes=CFG["num_classes"]).to(device)

    # Recalculate class weights for the criterion based on the current fold's training data
    fold_class_counts_tensor = torch.tensor(fold_label_counts.values, dtype=torch.float).to(device)
    fold_class_weights_criterion = 1. / fold_class_counts_tensor
    fold_class_weights_criterion = fold_class_weights_criterion / fold_class_weights_criterion.sum()
    criterion = nn.CrossEntropyLoss(weight=fold_class_weights_criterion)


    optimizer = optim.AdamW(model.parameters(), lr=CFG["lr"], weight_decay=CFG["weight_decay"])
    scaler = GradScaler() if device.type == 'cuda' else None

    # Training loop
    for epoch in range(CFG["epochs"]):
        print(f"--- Epoch {epoch+1}/{CFG['epochs']} ---")

        train_loss = train_one_epoch(model, train_loader, criterion, optimizer, device, scaler, mixup_alpha=0.2)
        val_loss, val_f1 = evaluate_model(model, val_loader, criterion, device)

        print(f"Train Loss: {train_loss:.4f}")
        print(f"Val Loss: {val_loss:.4f}, Val F1: {val_f1:.4f}")

        # Save best model and implement early stopping
        if val_f1 > best_f1:
            best_f1 = val_f1
            torch.save(model.state_dict(), best_model_path)
            print(f"Saved best model with F1: {best_f1:.4f}")
            epochs_without_improvement = 0 # Reset counter
        else:
            epochs_without_improvement += 1

        if epochs_without_improvement >= early_stopping_patience:
            print(f"Early stopping after {early_stopping_patience} epochs without improvement.")
            break # Stop training loop for the current fold


print(f"\nTraining finished. Best validation F1-score: {best_f1:.4f}")

===== Fold 1/5 =====


/tmp/ipython-input-3403317544.py:96: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler() if device.type == 'cuda' else None


--- Epoch 1/20 ---


/tmp/ipython-input-357726773.py:13: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast(enabled=scaler is not None):


Train Loss: 0.7261
Val Loss: 0.8317, Val F1: 0.5649
Saved best model with F1: 0.5649
--- Epoch 2/20 ---
Train Loss: 0.3939
Val Loss: 0.4035, Val F1: 0.8510
Saved best model with F1: 0.8510
--- Epoch 3/20 ---
Train Loss: 0.3265
Val Loss: 0.3339, Val F1: 0.8840
Saved best model with F1: 0.8840
--- Epoch 4/20 ---
Train Loss: 0.2769
Val Loss: 0.2435, Val F1: 0.9320
Saved best model with F1: 0.9320
--- Epoch 5/20 ---
Train Loss: 0.2908
Val Loss: 0.2373, Val F1: 0.9309
--- Epoch 6/20 ---
Train Loss: 0.2617
Val Loss: 0.2404, Val F1: 0.9345
Saved best model with F1: 0.9345
--- Epoch 7/20 ---
Train Loss: 0.3155
Val Loss: 0.2564, Val F1: 0.9376
Saved best model with F1: 0.9376
--- Epoch 8/20 ---
Train Loss: 0.2048
Val Loss: 0.1646, Val F1: 0.9603
Saved best model with F1: 0.9603
--- Epoch 9/20 ---
Train Loss: 0.2490
Val Loss: 0.1535, Val F1: 0.9670
Saved best model with F1: 0.9670
--- Epoch 10/20 ---
Train Loss: 0.2300
Val Loss: 0.1521, Val F1: 0.9675
Saved best model with F1: 0.9675
--- Epoch 1

In [19]:
# Load the best model
model = build_model(model_name=CFG["model_name"], num_classes=CFG["num_classes"]).to(device)
model.load_state_dict(torch.load(best_model_path))
model.eval()

# Prepare test data
test_files = scan_files(TESTD)
# Generate test IDs based on the number of test files and their naming convention
test_ids = [int(os.path.splitext(os.path.basename(f))[0]) for f in test_files]
test_ids = sorted(test_ids) # Ensure IDs are in order

test_dataset = AIKuthonDataset(test_files, labels=None, val_transform=val_transform) # No labels for test set
test_loader = DataLoader(
    test_dataset,
    batch_size=CFG["batch_size"],
    num_workers=CFG["num_workers"],
    pin_memory=True,
    shuffle=False,
)

# Perform inference
all_preds = []
with torch.no_grad():
    for images, _ in test_loader: # No labels in test_loader
        images = images.to(device)
        outputs = model(images)
        preds = torch.argmax(outputs, dim=1)
        all_preds.append(preds)

all_preds = torch.cat(all_preds).cpu().numpy()

# Create submission file
submission_df = pd.DataFrame({'id': test_ids, 'label': all_preds})
submission_df.to_csv('submission.csv', index=False)

print("Submission file 'submission.csv' created successfully.")
display(submission_df.head())

Submission file 'submission.csv' created successfully.


,id,label
0,1,4
1,2,3
2,3,5
3,4,1
4,5,1


## 태스크 완료

클래스 불균형 문제를 해결하기 위해 EfficientNet-B0 모델, dropout, mixup 기법을 사용하여 이미지 크기를 224x224로 조정하고 모델 학습 및 평가를 진행했습니다. 또한, 테스트 데이터에 대한 추론을 수행하고 Kaggle 제출 형식에 맞는 `submission.csv` 파일을 성공적으로 생성했습니다.

**주요 구현 내용:**

- 데이터셋 클래스 불균형 확인 및 WeightedRandomSampler를 사용한 오버샘플링 적용
- Albumentations를 활용한 데이터 증강 및 Mixup 구현
- EfficientNet-B0 모델 로드 및 클래스 수에 맞게 수정, Dropout 적용
- 클래스 가중치를 고려한 Cross-Entropy 손실 함수 및 AdamW 옵티마이저 정의
- F1-score를 평가 지표로 사용한 학습 및 평가 함수 구현
- Stratified K-Fold 교차 검증 및 조기 종료(Early Stopping) 적용
- 테스트 데이터셋에 대한 추론 및 제출 파일 생성

제출 파일인 `submission.csv`는 현재 디렉토리에 저장되었습니다. 이 파일을 Kaggle 대회에 제출하시면 됩니다.